In [1]:
from pyspark.ml.linalg import Vectors
from pyspark.mllib.linalg import SparseVector, Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel, RandomForest
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.feature import Normalizer
import time

In [2]:
# Read the dataset that was previously splitted due to jupyter's maximum file size issue
rawData = sc.textFile("./dataset/covertype.data")
# rawData = sc.textFile("*.csv")

def prepare(line):
    values = line.split(',')
    map(float, values)
    featureVector = Vectors.dense(values[:-1])
    # Decision tree labels varies from 0 to n-1
    label = float(values[-1])-1
    return LabeledPoint(label, featureVector)

def new_prepare(line):
    values = line.split(',')
    map(float, values)
    wilderness = values[10:14].index('1')
    soil = values[14:54].index('1')
    featureVector = Vectors.dense(values[0:10]+[wilderness,soil])
    # Decision tree labels varies from 0 to n-1
    label = float(values[-1])-1
    return LabeledPoint(label, featureVector)

data = rawData.map(lambda line : new_prepare(line))

trainData, cvData, testData = data.randomSplit([0.8, 0.1, 0.1])

trainData.cache()
cvData.cache()
testData.cache()

PythonRDD[4] at RDD at PythonRDD.scala:48

In [3]:
# Treino: trainData
# Teste: cvData
# Descobrir a configuração dos hyperparameters

start_time = time.time()


model = DecisionTree.trainClassifier(trainData, numClasses=7, categoricalFeaturesInfo={10: 4, 11: 40},
    impurity='entropy', maxDepth=20, maxBins=300)
predictions = model.predict(cvData.map(lambda x: x.features))
labelsAndPredictions = cvData.map(lambda lp: lp.label).zip(predictions) 
m = MulticlassMetrics(labelsAndPredictions)

print m.precision()
print("--- %s seconds ---" % (time.time() - start_time))

# 0.923581051717
# --- 132.968389988 seconds ---

/opt/spark/current/python/pyspark/mllib/evaluation.py:237: UserWarning: Deprecated in 2.0.0. Use accuracy.
  warnings.warn("Deprecated in 2.0.0. Use accuracy.")


0.926300752139
--- 146.624416113 seconds ---


In [4]:
# Treino: trainData + cvData
# Teste: testData
# Verificar a precisão da Decision Tree usando o conjunto de teste
start_time = time.time()


model = DecisionTree.trainClassifier(trainData.union(cvData), numClasses=7, categoricalFeaturesInfo={10: 4, 11: 40}, 
                                     impurity='entropy', maxDepth=20, maxBins=300)
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions) 
m = MulticlassMetrics(labelsAndPredictions)

print m.precision()
print("--- %s seconds ---" % (time.time() - start_time))

# 0.926020983265
# --- 170.648295164 seconds ---

0.926466802807
--- 134.466556072 seconds ---


In [5]:
# Treino: trainData + cvData
# Teste: testData
# Random Forest com 20 Decision Trees

start_time = time.time()


forest = RandomForest.trainClassifier(trainData.union(cvData), numClasses=7, categoricalFeaturesInfo={10: 4, 11: 40}, numTrees=20,
                                      featureSubsetStrategy='auto', impurity='entropy', maxDepth=30, maxBins=300)
predictions = forest.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions) 
k = MulticlassMetrics(labelsAndPredictions)
print k.precision()


print("--- %s seconds ---" % (time.time() - start_time))

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/current/python/lib/py4j-0.10.3-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/opt/spark/current/python/lib/py4j-0.10.3-src.zip/py4j/java_gateway.py", line 1040, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 49030)


Traceback (most recent call last):
  File "/usr/lib64/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib64/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib64/python2.7/SocketServer.py", line 649, in __init__
    self.handle()
  File "/opt/spark/current/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/opt/spark/current/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
EOFError
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43471)
Traceback (most recent call last):
  File "/opt/spark/current/python/lib/py4j-0.10.3-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.co

----------------------------------------


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:43471)

In [ ]:
# Treino: trainData + cvData
# Teste: testData
# Multilayer Perceptron com 1 camada oculta

# MLP não suporta feature categórica
data = rawData.map(lambda line : prepare(line))

trainData, cvData, testData = data.randomSplit([0.8, 0.1, 0.1])

trainData.cache()
cvData.cache()
testData.cache()

start_time = time.time()

trainDF = trainData.union(cvData).map(lambda x: (x.label, Normalizer().transform(x.features).asML())).toDF(["label", "features"])
mlp = MultilayerPerceptronClassifier(maxIter=100, layers=[54, 100, 7], blockSize=128, seed=123)
model = mlp.fit(trainDF)
testDF = testData.map(lambda x: (Normalizer().transform(x.features).asML(),)).toDF(["features"])

predictions = model.transform(testDF).select("prediction")
labelsAndPredictions = testData.map(lambda x: x.label).zip(predictions.rdd.map(lambda x: x[0])) 
m = MulticlassMetrics(labelsAndPredictions)
print(m.precision())

print("--- %s seconds ---" % (time.time() - start_time))

# 0.687774846086
# --- 329.418580055 seconds ---